###  Хакатон

In [1]:
import pandas as pd
from catboost import Pool, CatBoostClassifier
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split 

import nltk 
import re
import pymorphy2
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

from tqdm import tqdm

import sklearn.metrics as sk_met # для оценки модели 


%matplotlib inline

In [2]:
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('punkt')

In [3]:
data_path = "../data/russian_inappropriate_messages/"

In [8]:
def convert(data_path, file, new_file):
    file_name = os.path.join(data_path, file)
    cols = ["text", "inappropriate"]
    df = pd.read_csv(file_name, usecols=cols)
    df.dropna(inplace=True)
    mask = (df.inappropriate == 0.0) | (df.inappropriate == 1.0)
    df = df.loc[mask, ["text", "inappropriate"]]
    df["inappropriate"] = df["inappropriate"].astype("int")    
    new_file_name = os.path.join(data_path, new_file)
    df.to_csv(new_file_name, index=0)
    return

in_file = ["train.csv", "val.csv", "test.csv"]
out_file = ["train_new.csv", "val_new.csv", "test_new.csv"]
for file_in, file_out in zip(in_file, out_file):
    convert(data_path, file_in, file_out)

In [9]:
stop_words = set(stopwords.words('russian'))
print(stop_words)

{'без', 'нас', 'разве', 'впрочем', 'бы', 'мне', 'этот', 'теперь', 'тогда', 'хорошо', 'при', 'всегда', 'он', 'не', 'со', 'ничего', 'этом', 'между', 'и', 'этой', 'три', 'с', 'нее', 'чуть', 'по', 'под', 'потом', 'что', 'на', 'вдруг', 'все', 'себя', 'того', 'перед', 'вас', 'совсем', 'был', 'же', 'лучше', 'их', 'ну', 'есть', 'от', 'этого', 'ей', 'во', 'меня', 'больше', 'ведь', 'ее', 'всех', 'себе', 'ни', 'ж', 'том', 'если', 'то', 'ним', 'опять', 'она', 'нет', 'когда', 'вот', 'моя', 'другой', 'из', 'потому', 'нибудь', 'можно', 'где', 'они', 'всю', 'была', 'через', 'в', 'сейчас', 'даже', 'были', 'об', 'еще', 'только', 'чтобы', 'два', 'раз', 'тот', 'куда', 'о', 'много', 'да', 'чего', 'иногда', 'к', 'за', 'будет', 'зачем', 'тут', 'такой', 'было', 'тебя', 'вы', 'свою', 'никогда', 'но', 'быть', 'эти', 'чем', 'у', 'уж', 'ней', 'может', 'тоже', 'всего', 'для', 'или', 'после', 'сам', 'про', 'вам', 'один', 'ему', 'более', 'до', 'надо', 'конечно', 'его', 'я', 'какая', 'ты', 'почти', 'чтоб', 'так', 'им

### Приведение к нормальной форме (нижний регистр и лемматизация)

In [10]:
def lemmatized(df_train, text_col):
    # нормализация текста: приведение к нижнему регистру, удаление различных символов
    df_train[text_col] = df_train[text_col].str.lower()
    df_train[text_col] = df_train[text_col].str.replace(',', ' ')
    df_train[text_col] = df_train[text_col].str.replace('.', ' ')
    df_train[text_col] = df_train[text_col].str.replace('-', ' ')
    df_train[text_col] = df_train[text_col].str.replace(';', ' ')
    df_train[text_col] = df_train[text_col].str.replace(':', ' ')
    df_train[text_col] = df_train[text_col].str.replace('(', ' ')
    df_train[text_col] = df_train[text_col].str.replace(')', ' ')
    df_train[text_col] = df_train[text_col].str.replace('}', ' ')
    df_train[text_col] = df_train[text_col].str.replace('{', ' ')
    df_train[text_col] = df_train[text_col].str.replace('<', ' ')
    df_train[text_col] = df_train[text_col].str.replace('>', ' ')

    df_train[text_col] = df_train[text_col].str.replace('!', ' ')
    df_train[text_col] = df_train[text_col].str.replace(r'\d+', ' ')
    df_train[text_col] = df_train[text_col].str.replace(r'[\W]+', ' ')
    
    return df_train

# приведение токенов входящих в текст к нормальной форме
def norm(text, morph):  
    text_norm = ''  
    for token in nltk.word_tokenize(text):
        # print('token = ', token)
        token_norm = morph.parse(token)[0].normal_form
        if token_norm not in stop_words:        
            text_norm = text_norm + ' ' + token_norm
        # print('text_norm', text_norm)        
    return text_norm

def norm_all_df(df_train, text_col):
    # приведение к нормальной форме всех отзывов
    morph = pymorphy2.MorphAnalyzer()
    N = df_train.shape[0]
#     N = 100
    with tqdm(total=N) as progress_bar:    
        for i in range(N):
            #print('i = ', i)
            df_train.loc[i, text_col] = norm(df_train.loc[i, text_col], morph)
            progress_bar.update()
    return df_train

In [11]:
def lemmatized_data(data_path, file_in, file_out):
    df = pd.read_csv(os.path.join(data_path, file_in))    
    file_lemmatized = os.path.join(data_path, file_out)

    text_col = 'text' # имя колонки с текстом

    df_with_lemm = lemmatized(df, text_col)
    df_with_lemm = norm_all_df(df_with_lemm, text_col)
    df_with_lemm.dropna(inplace=True)
    df_with_lemm.to_csv(file_lemmatized, sep=";", index=False)
    return

in_file = ["train_new.csv", "val_new.csv", "test_new.csv"]
out_file = ["train_lemm.csv", "val_lemm.csv", "test_lemm.csv"]
for file_in, file_out in zip(in_file, out_file):
    lemmatized_data(data_path, file_in, file_out)

/tmp/ipykernel_116421/4080072856.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_train[text_col] = df_train[text_col].str.replace('.', ' ')
/tmp/ipykernel_116421/4080072856.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_train[text_col] = df_train[text_col].str.replace('(', ' ')
/tmp/ipykernel_116421/4080072856.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_train[text_col] = df_train[text_col].str.replace(')', ' ')
/tmp/ipykernel_116421/4080072856.py:11: FutureWarning: The default value of regex wi

In [14]:
# df = pd.read_csv(os.path.join(data_path, "train_lemm.csv"), sep=";")
# df.info()

In [15]:
# df.dropna(inplace=True)
# df.info()

In [19]:
def drop_na(data_path, file):
    file_name = os.path.join(data_path, file)
    df = pd.read_csv(file_name, sep=";")
    df.dropna(inplace=True)
    df.to_csv(file_name, index=False, sep=";")
    
    return
    
for file in ["train_lemm.csv", "val_lemm.csv", "test_lemm.csv"]:
    drop_na(data_path, file)
    

### Обучим

In [20]:
target_col = "inappropriate"
train = pd.read_csv(os.path.join(data_path, "train_lemm.csv"), sep=";")
train.head()

,text,inappropriate
0,думать левиафан это медленный страхоебин numb...,1
1,напоминать пора искать актис невзрослый прон,1
2,курить год пятнадцать никакой проблема кроме ...,1
3,окей провести парад гетеросексуал гей ущемлят...,1
4,напоминать ватник сша общий налог весь number...,1


In [25]:
val = pd.read_csv(os.path.join(data_path, "val_lemm.csv"), sep=";")
val.head(3)

,text,inappropriate
0,это ширина длина член number диаметр смочь но...,1
1,красава др хаебуса сделать,1
2,судебный атака обнулить внезапно умереть коро...,1


In [26]:
train.inappropriate.value_counts()

0    63058
1    21824
Name: inappropriate, dtype: int64

In [27]:
val.inappropriate.value_counts()

0    7886
1    2716
Name: inappropriate, dtype: int64

In [31]:
def fit_catboost(
    X_train, 
    X_test, 
    y_train, 
    y_test, 
    catboost_params = {},
    verbose = 100
):
    learn_pool = Pool(
        X_train, 
        y_train, 
        text_features=["text"], 
        feature_names=["text"]
    )
    test_pool = Pool(
        X_test, 
        y_test, 
        text_features=["text"],
        feature_names=["text"]
    )
    catboost_default_params = {
        'iterations': 5000,
        'learning_rate': 0.015,
        'eval_metric': 'F1',
        'task_type': 'GPU',
        'use_best_model': True
    }
    catboost_default_params.update(catboost_params)
    
    model = CatBoostClassifier(**catboost_default_params)
    model.fit(learn_pool, eval_set=test_pool, verbose=verbose)
    return model



In [29]:
X_train = train[["text"]]
X_val = val[["text"]]
y_train = train[target_col]
y_val = val[target_col]

In [32]:
# X_train, X_val, y_train, y_val = train_test_split(
#     train[["text"]],
#     train[target_col],
#     test_size=0.3, 
#     stratify=train[target_col],
#     random_state=42
# )
cat_boost_model = fit_catboost(X_train, X_val, y_train, y_val)

0:	learn: 0.2265108	test: 0.2388729	best: 0.2388729 (0)	total: 10.4ms	remaining: 52.1s
100:	learn: 0.4910872	test: 0.5166253	best: 0.5174721 (94)	total: 829ms	remaining: 40.2s
200:	learn: 0.5085406	test: 0.5326805	best: 0.5326805 (198)	total: 1.61s	remaining: 38.6s
300:	learn: 0.5208752	test: 0.5422007	best: 0.5426844 (297)	total: 2.4s	remaining: 37.5s
400:	learn: 0.5290133	test: 0.5494717	best: 0.5494717 (400)	total: 3.15s	remaining: 36.1s
500:	learn: 0.5372518	test: 0.5526127	best: 0.5526127 (494)	total: 3.89s	remaining: 34.9s
600:	learn: 0.5460881	test: 0.5602652	best: 0.5602652 (584)	total: 4.63s	remaining: 33.9s
700:	learn: 0.5505187	test: 0.5631480	best: 0.5631480 (697)	total: 5.37s	remaining: 33s
800:	learn: 0.5553181	test: 0.5660023	best: 0.5660023 (800)	total: 6.12s	remaining: 32.1s
900:	learn: 0.5598549	test: 0.5684947	best: 0.5684947 (898)	total: 6.91s	remaining: 31.4s
1000:	learn: 0.5625441	test: 0.5707635	best: 0.5707635 (992)	total: 7.66s	remaining: 30.6s
1100:	learn: 0.5

### Проверим f1 на данных которые модель не видела

In [34]:
test = pd.read_csv(os.path.join(data_path, "test_lemm.csv"), sep=";")
test.head()

,text,inappropriate
0,тупой хотя проверить насколько удачливпопробо...,1
1,это начаться война террористомить америка,1
2,ой никак понять хотеть денюх хотеть большой ж...,1
3,это плохой пес пасха это например праздновани...,1
4,насчёт шизофрения считать это плохой подарок ...,1


In [35]:
X_test, y_test = test[["text"]], test[target_col]
X_test.head()

,text
0,тупой хотя проверить насколько удачливпопробо...
1,это начаться война террористомить америка
2,ой никак понять хотеть денюх хотеть большой ж...
3,это плохой пес пасха это например праздновани...
4,насчёт шизофрения считать это плохой подарок ...


In [36]:
y_pred = cat_boost_model.predict(X_test)
print(
    'F1-score на тестовой выборке: {:.3f} \n'
    .format(
        sk_met.f1_score(
            y_test, 
            y_pred, 
            average = 'macro')
    )
)

F1-score на тестовой выборке: 0.743 



In [37]:
cat_boost_model.save_model("hack_model_russian")